In [85]:
import fasttext

"""
The training file should be this format:

__label__pp this is my text
__label__psoe this is also my text
"""

'\nThe training file should be this format:\n\n__label__pp this is my text\n__label__psoe this is also my text\n'

In [116]:
import pandas as pd
import numpy as np
import re
import emoji

#Function to get all emojis in a text.
def split_emojis(text):

    emoji_list = []
    data = re.findall(r'\X', text)
    for word in data:
        if any(char in emoji.UNICODE_EMOJI for char in word):
            emoji_list.append(word)

    return emoji_list

#Function to clean the text of a tweet.
def clean_tweet(text):
    clean_text = re.sub("@.*:", "", text) #This do not improve performance.
    clean_text = re.sub("@.* ", "", clean_text)
    clean_text = re.sub("RT", "", clean_text)
    clean_text = re.sub("\n", " ", clean_text)
    #Separate emojis from words.
    """
    sep_emojis_text = ""
    for c in clean_text:
        if c in emoji.UNICODE_EMOJI:
            sep_emojis_text+=' '+c+' '
        else:
            sep_emojis_text+=c
     #This do not improve performance.
    """
    return clean_text

#Read and clean data for each party.

#Podemos.
up_cleaned_data = pd.DataFrame()
up_df = pd.read_csv ('up.csv')
up_cleaned_data["tweet"] = up_df["text"].apply(clean_tweet)
up_cleaned_data["label"] = "__label__up"
print(len(up_df))
#Psoe.
psoe_cleaned_data = pd.DataFrame()
psoe_df = pd.read_csv ('psoe.csv')
psoe_cleaned_data["tweet"] = psoe_df["text"].apply(clean_tweet)
psoe_cleaned_data["label"] = "__label__psoe"
print(len(psoe_df))
#CS.
cs_cleaned_data = pd.DataFrame()
cs_df = pd.read_csv ('cs.csv')
cs_cleaned_data["tweet"] = cs_df["text"].apply(clean_tweet)
cs_cleaned_data["label"] = "__label__cs"
print(len(cs_df))
#PP.
pp_cleaned_data = pd.DataFrame()
pp_df = pd.read_csv ('pp.csv')
pp_cleaned_data["tweet"] = pp_df["text"].apply(clean_tweet)
pp_cleaned_data["label"] = "__label__pp"
print(len(pp_df))
#Vox.
vox_cleaned_data = pd.DataFrame()
vox_df = pd.read_csv ('vox.csv')
vox_cleaned_data["tweet"] = vox_df["text"].apply(clean_tweet)
vox_cleaned_data["label"] = "__label__vox"
print(len(vox_df))

#Concat all parties data.
data = pd.concat([up_cleaned_data, psoe_cleaned_data, cs_cleaned_data, pp_cleaned_data, vox_cleaned_data])

4979
4991
5996
4955
4967


In [117]:
#Split data in training and test sets.
import csv
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data["tweet"], data["label"], test_size=0.15, random_state=42)

training = pd.concat([X_train,y_train], axis=1)
display(training)

test = pd.concat([X_test,y_test], axis=1)
display(test)

#Create training file.
training[training.columns.values[::-1]].to_csv('data.train', index=False, sep=' ', header=False, quoting=csv.QUOTE_NONE, quotechar="", escapechar=" ")
#data.to_csv('data.train', index=False, sep=' ', header=False, quoting=csv.QUOTE_NONE, quotechar="", escapechar=" ")

#Create test file.
test[test.columns.values[::-1]].to_csv('data.test', index=False, sep=' ', header=False, quoting=csv.QUOTE_NONE, quotechar="", escapechar=" ")

,tweet,label
4592,⚒💚⚒ #LevantandoEspaña,__label__vox
4908,❌ ¡La homosexualidad nunca ha sido una enfer...,__label__cs
5514,☀️ ¡Buenos días y #FelizMartes! 📻 Empieza la...,__label__cs
2807,//t.co/O3gMgMHob5,__label__psoe
2271,VA.,__label__vox
...,...,...
654,//t.co/f9A4j7YuSR,__label__vox
411,Este año celebramos el #12deOctubre reafirma...,__label__psoe
860,Esta mañana ha llegado el primer SÍ a los Pr...,__label__up
5825,🔵 Unos prefieren ver si solos sacan…,__label__cs


,tweet,label
1783,A punto de empezar el encuentro telemático con...,__label__cs
51,☝️Ni una semana ha tardado la vicelehendakar...,__label__cs
1746,"🗞️ ""Son unos PGE necesarios, progresistas, ...",__label__psoe
2203,entiendo,__label__pp
3027,🤗,__label__up
...,...,...
853,Al Gore tardó 30 días en reconocer su derrot...,__label__vox
4130,💜 Ideas. 💜 Principios. 💜 Políticas. 💜 𝐅𝐄𝐌𝐈𝐍𝐈𝐒...,__label__psoe
378,Día de la Hispanidad https://t.co/Vd6lmFmZOB,__label__vox
3589,//t.co/TOJktMsX34,__label__up


In [118]:
import fasttext
# Load input training set.

#By default, fastText sees each training example only five times during training, which is pretty small. The number of times each examples is seen (also known as the number of epochs), can be increased using the -epoch option
model = fasttext.train_supervised(input="data.train", epoch=25)

In [119]:
#To predict one instance.
print( model.predict("Franco deberia volver a vivir") )
print( model.predict("Los españoles son los mejores 💙") )
print( model.predict("Los españoles son los mejores 💚") )
print( model.predict("España es una gran nacion") )
print( model.predict("Pablo Iglesias es el mejor") )
print( model.predict("Pablo Iglesias es un mierdas") )
print( model.predict("Putos progres de mierda") )
print( model.predict("Viva la guardia civil") )
print( model.predict("Igualdad salarial para las mujeres!") )
print( model.predict("La españa moderada") )
print( model.predict("La mujer a trabajar") )
print( model.predict("Viva la fuerza naranja") )

(('__label__vox',), array([0.94536889]))
(('__label__pp',), array([0.98429704]))
(('__label__vox',), array([0.99206376]))
(('__label__cs',), array([0.96874827]))
(('__label__up',), array([0.65534759]))
(('__label__up',), array([0.98422652]))
(('__label__vox',), array([1.00000691]))
(('__label__cs',), array([0.95531917]))
(('__label__up',), array([0.90752602]))
(('__label__cs',), array([0.84613037]))
(('__label__cs',), array([0.85025179]))
(('__label__cs',), array([0.80714887]))


In [120]:
#To test the test set. (number of samples, precision, recall)
model.test("data.test")

precision = model.test("data.test")[1]
recall = model.test("data.test")[2]

F_score = 2*(precision * recall)/(precision + recall)

print("Precision:", precision)
print("Recall:", recall)
print("Fscore:", F_score)

Precision: 0.49922760041194647
Recall: 0.49922760041194647
Fscore: 0.49922760041194647


In [121]:
model.save_model("fastTextModel.model")